In [1]:
pip install lumibot

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     --- ------------------------------------ 0.3/3.0 MB ? eta -:--:--
     --- ------------------------------------ 0.3/3.0 MB ? eta -:--:--
     --- ------------------------------------ 0.3/3.0 MB ? eta -:--:--
     --- ------------------------------------ 0.3/3.0 MB ? eta -:--:--
     ------ --------------------------------- 0.5/3.0 MB 299.6 kB/s eta 0:00:09
     ------ --------------------------------- 0.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.26.4 which is incompatible.


In [5]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from datetime import datetime
from alpaca_trade_api import REST
from timedelta import Timedelta
from finbert_utils import estimate_sentiment

API_KEY = "#"
API_SECRET = "#"
BASE_URL = "#"

ALPACA_CREDS = {
    "API_KEY":API_KEY,
    "API_SECRET":API_SECRET,
    "PAPER":True
}

class MLTrader(Strategy): 
    def initialize(self, symbol:str="SPY", cash_at_risk:float=.5): 
        self.symbol = symbol
        self.sleeptime = "24H" 
        self.last_trade = None 
        self.cash_at_risk = cash_at_risk
        self.api = REST(base_url=BASE_URL, key_id=API_KEY, secret_key=API_SECRET)

    def position_sizing(self): 
        cash = self.get_cash() 
        last_price = self.get_last_price(self.symbol)
        quantity = cash * self.cash_at_risk // last_price
        return cash, last_price, quantity

    def get_dates(self): 
        today = self.get_datetime()
        three_days_prior = today - Timedelta(days=7)
        return today.strftime('%Y-%m-%d'), three_days_prior.strftime('%Y-%m-%d')

    def get_sentiment(self): 
        today, three_days_prior = self.get_dates()
        news = self.api.get_news(symbol=self.symbol, 
                                 start=three_days_prior, 
                                 end=today) 
        news = [ev.__dict__["_raw"]["headline"] for ev in news]
        probability, sentiment = estimate_sentiment(news)
        return probability, sentiment 

    def on_trading_iteration(self):
        cash, last_price, quantity = self.position_sizing() 
        probability, sentiment = self.get_sentiment()
        position = self.get_position(self.symbol)
        if cash > last_price: 
            if sentiment == "positive" and probability > .999: 
                if self.last_trade == "sell": 
                    self.sell_all() 
                order = self.create_order(
                    self.symbol, 
                    quantity, 
                    "buy", 
                    type="bracket", 
                    take_profit_price=last_price*1.20, 
                    stop_loss_price=last_price*.95
                )
                self.submit_order(order) 
                self.last_trade = "buy"
            elif sentiment == "negative" and probability > .999: 
                if self.last_trade == "buy": 
                    self.sell_all() 
                order = self.create_order(
                    self.symbol, 
                    quantity, 
                    "sell", 
                    type="bracket", 
                    take_profit_price=last_price*.8, 
                    stop_loss_price=last_price*1.05
                )
                self.submit_order(order) 
                self.last_trade = "sell"

start_date = datetime(2025,1,1)
end_date = datetime(2025,3,1) 
broker = Alpaca(ALPACA_CREDS) 
strategy = MLTrader(name='mlstrat', broker=broker, 
                    parameters={"symbol":"SPY", 
                                "cash_at_risk":.5})
strategy.backtest(
    YahooDataBacktesting, 
    start_date, 
    end_date, 
    parameters={"symbol":"SPY", "cash_at_risk":.5}
)




'''
start_date = datetime(2025,1,15)
end_date = datetime(2025,2,28)

class MLTrader(Strategy):
    def initialize(self, symbol:str="SPY", cash_at_risk:float=.5):
        self.symbol = symbol
        self.sleeptime = "24H"
        self.last_trade = None
        self.cash_at_risk = cash_at_risk
        self.api = REST(base_url = BASE_URL, key_id = API_KEY, secret_key = API_SECRET)
        
    def position_sizing(self):
        cash = self.get_cash()
        last_price = self.get_last_price(self.symbol)
        quantity = round(cash * self.cash_at_risk / last_price, 0)
        return cash, last_price, quantity

    def get_dates(self):
        today = self.get_datetime()
        three_days_prior = today - Timedelta(days=3)
        return today.strftime('%Y-%m-%d'), three_days_prior.strftime('%Y-%m-%d')
        
    def get_news(self):
        today, three_days_prior = self.get_dates()
        news = self.api.get_news(symbol = self.symbol, start = three_days_prior, end = today)
        news = [ev.__dict__["_raw"]["headline"] for ev in news]
        return news
        
    def on_trading_iteration(self):
        cash, last_price, quantity = self.position_sizing()
    
        # Check if we have an open position
        position = self.get_position(self.symbol)

        if position is None and cash > last_price:  # No position, enough cash → Buy
            news = self.get_news()
            print(news)
            order = self.create_order(
                self.symbol,
                10,
                "buy",
                type="bracket",
                take_profit_price=last_price * 1.20,
                stop_loss_price=last_price * 0.95,
            )
            self.submit_order(order)
            self.last_trade = "buy"
    
        elif position and self.last_trade == "buy":  # Position exists → Sell if conditions met
            # Example: Exit after reaching a profit target or stop loss
            if last_price >= position.entry_price * 1.20 or last_price <= position.entry_price * 0.95:
                order = self.create_order(self.symbol, position.quantity, "sell", type="market")
                self.submit_order(order)
                self.last_trade = None  # Reset for the next trade

broker = Alpaca(ALPACA_CREDS)
strategy = MLTrader(name='mlstrat',broker=broker, 
                    parameters={"symbol":"SPY",
                               "cash_at_risk":.5}
                   )
strategy.backtest(
    YahooDataBacktesting,
    start_date,
    end_date,
    parameters={"symbol":"SPY","cash_at_risk":.5}
)
'''
#lumibot - Easy algo trading framework
#alpaca-trade-api-python - Get news and place trades to broker
#datetime - For date formatting
#timedelta - Calculating time differences
#torch - pytorch framework for using AI/ML
#transformers - Load up finance deep learning model

Progress |██████████████████████████████████████████████████████████████████| 100.00%  [Elapsed: 0:00:20 ETA: 0:00:00] 

'\nstart_date = datetime(2025,1,15)\nend_date = datetime(2025,2,28)\n\nclass MLTrader(Strategy):\n    def initialize(self, symbol:str="SPY", cash_at_risk:float=.5):\n        self.symbol = symbol\n        self.sleeptime = "24H"\n        self.last_trade = None\n        self.cash_at_risk = cash_at_risk\n        self.api = REST(base_url = BASE_URL, key_id = API_KEY, secret_key = API_SECRET)\n        \n    def position_sizing(self):\n        cash = self.get_cash()\n        last_price = self.get_last_price(self.symbol)\n        quantity = round(cash * self.cash_at_risk / last_price, 0)\n        return cash, last_price, quantity\n\n    def get_dates(self):\n        today = self.get_datetime()\n        three_days_prior = today - Timedelta(days=3)\n        return today.strftime(\'%Y-%m-%d\'), three_days_prior.strftime(\'%Y-%m-%d\')\n        \n    def get_news(self):\n        today, three_days_prior = self.get_dates()\n        news = self.api.get_news(symbol = self.symbol, start = three_days_pri

In [9]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from datetime import datetime, timedelta
from alpaca_trade_api import REST
from timedelta import Timedelta
from finbert_utils import estimate_sentiment
import numpy as np

API_KEY = "#"
API_SECRET = "#"
BASE_URL = "#"

ALPACA_CREDS = {
    "API_KEY": API_KEY,
    "API_SECRET": API_SECRET,
    "PAPER": True
}

class MLTrader(Strategy):
    def initialize(self, symbol="SPY", cash_at_risk=0.5):
        self.symbol = symbol
        self.sleeptime = "24H"
        self.last_trade = None
        self.cash_at_risk = cash_at_risk
        self.api = REST(base_url=BASE_URL, key_id=API_KEY, secret_key=API_SECRET)

    def get_volatility(self, symbol, period=20):
        # Try fetching historical prices with the lookback period
        bars = self.get_historical_prices(symbol, length=20, timestep="day")  # Adjust length as needed


        # Debugging: Print bars info
        print("Bars:", bars)
        print("Type of bars:", type(bars))

        # Ensure bars is valid
        if bars is None or not hasattr(bars, "__iter__") or not list(bars):
            print("Error: bars is empty or not iterable")
            return None  

        # Convert to DataFrame if possible
        if hasattr(bars, "to_dataframe"):
            bars = bars.to_dataframe()
            if bars.empty:
                print("Bars converted to empty DataFrame")
                return None

        # Assuming bars is a DataFrame, calculate volatility
        returns = bars["close"].pct_change().dropna()
        volatility = returns.std()

        return volatility









    def position_sizing(self):
        cash = self.get_cash()
        last_price = self.get_last_price(self.symbol)
        volatility = self.get_volatility(self.symbol, period=20)

        # Adjust position size based on volatility
        adjusted_cash_at_risk = min(0.5, max(0.1, 1 / (volatility + 1e-6)))
        quantity = cash * adjusted_cash_at_risk // last_price
        return cash, last_price, quantity

    def get_dates(self):
        today = self.get_datetime()
        seven_days_prior = today - Timedelta(days=7)
        return today.strftime('%Y-%m-%d'), seven_days_prior.strftime('%Y-%m-%d')

    def get_sentiment(self):
        today, seven_days_prior = self.get_dates()
        news = self.api.get_news(symbol=self.symbol, start=seven_days_prior, end=today)
        
        headlines = [ev.__dict__["_raw"]["headline"] for ev in news] if news else []
        
        if not headlines:  
            return 0, "neutral"

        probabilities, sentiments = estimate_sentiment(headlines)
        avg_sentiment = np.mean(probabilities)

        return avg_sentiment, "positive" if avg_sentiment > 0.7 else "negative" if avg_sentiment < 0.3 else "neutral"

    def on_trading_iteration(self):
        cash, last_price, quantity = self.position_sizing()
        probability, sentiment = self.get_sentiment()

        # Get moving averages for trend confirmation
        sma_50 = self.get_sma(self.symbol, 50)
        sma_200 = self.get_sma(self.symbol, 200)
        uptrend = last_price > sma_50 > sma_200
        downtrend = last_price < sma_50 < sma_200

        position = self.get_position(self.symbol)
        entry_price = position.entry_price if position else None

        # Buy Logic
        if cash > last_price and sentiment == "positive" and probability > 0.7 and uptrend:
            if self.last_trade == "sell":
                self.sell_all()

            order = self.create_order(
                self.symbol, quantity, "buy", type="trailing_stop", trail_percent=5
            )
            self.submit_order(order)
            self.last_trade = "buy"

        # Sell Logic
        elif sentiment == "negative" and probability > 0.7 and downtrend:
            if self.last_trade == "buy":
                self.sell_all()

            order = self.create_order(
                self.symbol, quantity, "sell", type="trailing_stop", trail_percent=5
            )
            self.submit_order(order)
            self.last_trade = "sell"

        # Stop loss check
        if position and entry_price:
            if last_price >= entry_price * 1.20 or last_price <= entry_price * 0.95:
                self.sell_all()

# Backtesting
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 3, 1)

broker = Alpaca(ALPACA_CREDS)
strategy = MLTrader(name="mlstrat", broker=broker, parameters={"symbol": "SPY", "cash_at_risk": 0.5})

strategy.backtest(
    YahooDataBacktesting,
    start_date,
    end_date,
    parameters={"symbol": "SPY", "cash_at_risk": 0.5}
)


Bars:                                  open        high         low       close  \|   2.37%  [Elapsed: 0:00:01 ETA: 0:01:14] 
Date                                                                        
2024-12-03 16:00:00-05:00  603.390015  604.159973  602.340027  603.909973   
2024-12-04 16:00:00-05:00  605.630005  607.909973  604.950012  607.659973   
2024-12-05 16:00:00-05:00  607.659973  608.479980  606.299988  606.659973   
2024-12-06 16:00:00-05:00  607.440002  609.070007  607.020020  607.809998   
2024-12-09 16:00:00-05:00  607.690002  607.859985  604.080017  604.679993   
2024-12-10 16:00:00-05:00  605.369995  605.799988  602.130005  602.799988   
2024-12-11 16:00:00-05:00  605.780029  608.429993  605.500000  607.460022   
2024-12-12 16:00:00-05:00  606.580017  607.159973  604.330017  604.330017   
2024-12-13 16:00:00-05:00  606.400024  607.130005  602.809998  604.210022   
2024-12-16 16:00:00-05:00  606.000000  607.780029  605.210022  606.789978   
2024-12-17 16:00:00-05:00  

Exception in thread MLTrader:
Traceback (most recent call last):
  File "C:\Users\rocha\anaconda3\envs\py310_env\lib\site-packages\lumibot\strategies\strategy_executor.py", line 1059, in run
    self._run_trading_session()
  File "C:\Users\rocha\anaconda3\envs\py310_env\lib\site-packages\lumibot\strategies\strategy_executor.py", line 999, in _run_trading_session
    self._on_trading_iteration()
  File "C:\Users\rocha\anaconda3\envs\py310_env\lib\site-packages\lumibot\strategies\strategy_executor.py", line 306, in func_output
    result = func_input(self, *args, **kwargs)
  File "C:\Users\rocha\anaconda3\envs\py310_env\lib\site-packages\lumibot\strategies\strategy_executor.py", line 331, in func_output
    result = func_input(self, *args, **kwargs)
  File "C:\Users\rocha\anaconda3\envs\py310_env\lib\site-packages\lumibot\strategies\strategy_executor.py", line 466, in _on_trading_iteration
    raise e
  File "C:\Users\rocha\anaconda3\envs\py310_env\lib\site-packages\lumibot\strategies\st

2025-03-08 22:22:27,523: ERROR: Not enough data to create a tearsheet, at least 2 days of data are required. Skipping


{}